In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('train_dummy.csv')
test = pd.read_csv('test_dummy.csv')
target = pd.read_csv('label.csv')
df = pd.concat([train, test], axis=0)


'''
#is the applicant the only breadwinner in their household?
df['Only_Earner'] = df['CoapplicantIncome'].apply(lambda x : 0 if x==0 else 1)
#is the applicant the primary breadwinner in their household?
df['Primary_Earner'] = (df['ApplicantIncome'] > df['CoapplicantIncome'])
#what is the applicant's contribution to the household income
df['Applicant_Earning_Share'] = df['ApplicantIncome'] / df['Total_Income']
#what Fraction of the applicant's income is the loan amount?
df['Loan_Fraction'] = df['LoanAmount'] / df['Total_Income']


df['Primary_Earner'] = df['Primary_Earner'].map({True:1, False:0})

def per_capita_income(x):
    ctr = 1
    if x['Married'] == 1:
        ctr += 1
    if x['Dependents_1'] == 1:
        ctr += 1
    elif x['Dependents_3+'] == 1:
        ctr += 3
    elif x['Dependents_0'] == 0:
        ctr += 2
    return x['Total_Income'] / ctr

df['Per_Capita_Income'] = 0
per_capita = []
ptr=0
for item in df['Per_Capita_Income']:
    temp = per_capita_income(df.iloc[ptr, :])
    per_capita.append(temp)
    ptr += 1
df['Per_Capita_Income'] = per_capita
'''

# MACHINE LEARNING

In [ ]:
from sklearn.linear_model import LogisticRegression, LassoCV, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score
from sklearn.decomposition import PCA
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.13,shuffle=True)

lr = LogisticRegression()
sgdc = SGDClassifier()
ada = AdaBoostClassifier()
bag = BaggingClassifier()
etc = ExtraTreeClassifier()
gbc = GradientBoostingClassifier()
rfc = RandomForestClassifier()
gnb = GaussianNB()
bnb = BernoulliNB()
knn = KNeighborsClassifier()
xgb = XGBClassifier()
xgbr = XGBRFClassifier()
lgb = LGBMClassifier()
cat = CatBoostClassifier()

models = [lr, ada, sgdc, bag, etc, gbc, rfc, 
           knn, gnb, bnb, xgb, xgbr, lgb, cat]


In [ ]:
data = [x_train, y_train, x_test, y_test]

In [ ]:
def run_model(model, data):
    model.fit(data[0], data[1])
    pred = model.predict(data[2])
    score = accuracy_score(data[3], pred,)
    return score

In [ ]:
model_names = []
model_performances = []

for model in models:
    perf = run_model(model, data)
    model_performances.append(perf)
    model_names.append(str(model.get_params).split('(')[0].split()[-1])
    print(model_names[-1], model_performances[-1])
    

### Tuning the best performing models

In [ ]:
#sns.barplot(model_performances, model_names)

In [ ]:
gbc.fit(x, y)
pred = gbc.predict(X)


sub = pd.read_csv("sample_submission.csv")
sub['Loan_Status'] = pred
sub.to_csv('Submission_13.csv', index=False)

In [ ]:
def gridsearch(model, par):
    grid_search = GridSearchCV(model, par, verbose=3, scoring='accuracy')
    grid_search.fit(x_train, y_train)
    print('Best Parameters: ',grid_search.best_params_)
    print('Best Score: ',grid_search.best_score_)
    print('Accuracy Score = ',run_model(grid_search.best_estimator_, data))

In [ ]:
AdaBoostClassifier()
params_ada = {'n_estimators':[10,50,100,500,1000,10000], 'learning_rate':[0.001,0.005,0.01,0.05]}
gridsearch(ada, params_ada)

In [ ]:
run_model(AdaBoostClassifier(learning_rate=0.5, n_estimators=500), data)

In [ ]:
run_model(ada, data)

In [ ]:
GradientBoostingClassifier()
params_gbc = {'n_estimators':[10,50,100,500,1000], 'learning_rate':[0.01,0.1,0.5,0.9,1], 'max_depth':[3,5,7,10,15],
              'tol':[0.00001,0.0001,0.001,0.01,0.1,0.5]}
gridsearch(gbc, params_gbc)

In [ ]:
XGBRFClassifier()

params_xgbrfc = {'booster':['gbtree','gblinear','dart'], 'n_estimators':[10,50,100,500,1000,5000],

                 'max_depth':[3,5,7,9,13,15], 'learning_rate':[0.01,0.1,0.5,1,1.5]}
gridsearch(xgbr, params_xgbrfc)

In [ ]:
cat_feats = ['Gender', 'Married', 'Education', 'Self_Employed', 'Credit_History',
        'Dependents_0', 'Dependents_1', 'Dependents_3+',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Only_Earner', 'Primary_Earner']
fn = CatBoostClassifier(cat_features=cat_feats)
params_cat ={'n_estimators':[100,500], 'depth':[3,5,7,10,15],
            'learning_rate':[0.01,0.1,0.5,0.7], 'l2_leaf_reg':[3,7,12,20]}
gridsearch(fn, params_cat)

In [ ]:
learn = []
per = []
for i in range(1, 100):
    fn = CatBoostClassifier(cat_features=cat_feats, learning_rate=i/100)
    per.append(run_model(fn, data))
    learn.append(i)
plt.plot(learn, per)

In [ ]:
fn = CatBoostClassifier(cat_features=cat_feats, learning_rate=0.46)
run_model(fn, data)


In [ ]:
fn.fit(x, y)
pred = fn.predict(X)


sub = pd.read_csv("sample_submission.csv")
sub['Loan_Status'] = pred
sub.to_csv('Submission_17.csv', index=False)

In [ ]:
x['Credit_History'] = x['Credit_History'].apply(lambda x: str(x))
X['Credit_History'] = X['Credit_History'].apply(lambda x: str(x))


In [ ]:
fn.fit(x, y, cat_features=cat_feats)
pred = fn.predict(X)


sub = pd.read_csv("sample_submission.csv")
sub['Loan_Status'] = pred
sub.to_csv('Submission_17.csv', index=False)

In [ ]:
cat_feats = ['Gender', 'Married', 'Education', 'Self_Employed', 'Credit_History',
        'Dependents_0', 'Dependents_1', 'Dependents_3+',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Only_Earner', 'Primary_Earner']
x_train['Credit_History'] = x_train['Credit_History'].apply(lambda x: str(x))
x_test['Credit_History'] = x_test['Credit_History'].apply(lambda x: str(x))

cat.fit(x_train, y_train, cat_features=cat_feats)
p = cat.predict(x_test)
print('\nAccuracy = ', accuracy_score(y_test, p))

In [ ]:
cat.fit(x, y, cat_features=cat_feats)
pred = cat.predict(X)


sub = pd.read_csv("sample_submission.csv")
sub['Loan_Status'] = pred
sub.to_csv('Submission_14.csv', index=False)

In [ ]:

bnb.fit(x, y,)
pred = bnb.predict(X)


sub = pd.read_csv("sample_submission.csv")
sub['Loan_Status'] = pred
sub.to_csv('Submission_16.csv', index=False)